In [ ]:
import os
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.decomposition import PCA
from scipy.linalg import pinv
from PIL import Image
from tensorflow.keras.models import Model
import joblib
import numpy as np
import os
from numpy.linalg import pinv
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from scipy.spatial import distance
import os
import shutil
import random
from tqdm import tqdm  # Import tqdm for progress bars



print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [4]:
import os
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array


data_dir = '/content/drive/My Drive/CXR_modified/PadChest'

# List of image paths
image_paths = [os.path.join(data_dir, fname) for fname in os.listdir(data_dir) if fname.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Preprocessing function to load and process each image
def process_image(image_path):
    # Load image (adjust target_size if needed)
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img /= 255.0  # Rescale pixel values
    return img

# Preprocessing images
images = np.array([process_image(path) for path in image_paths])

# Splitting the data into training and validation sets
train_images, val_images = train_test_split(images, test_size=0.1, random_state=42)

# Now, 'train_images' and 'val_images' are your training and validation datasets


KeyboardInterrupt: 

In [ ]:
print(len(train_images))
print(len(val_images))

import matplotlib.pyplot as plt

def display_images(images, title):
    """ Display a sample of images """
    plt.figure(figsize=(12, 8))

    for i in range(9):  # Display first 9 images
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.axis('off')

    plt.suptitle(title)
    plt.show()

# Displaying some images from the training set
display_images(train_images, "Sample Images from Training Set")

# Displaying some images from the validation set
display_images(val_images, "Sample Images from Validation Set")


In [ ]:
# Function to display random images from a directory
def display_random_images_from_directory(directory, num_images=5):
    image_names = [name for name in os.listdir(directory) if name.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]
    random_images = random.sample(image_names, min(num_images, len(image_names)))

    plt.figure(figsize=(10, 10))
    for i, image_name in enumerate(random_images):
        img_path = os.path.join(directory, image_name)
        img = Image.open(img_path).convert('L')
        img = img.resize((256, 256), Image.ANTIALIAS)
        plt.subplot(1, 5, i + 1)
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        plt.title(image_name[:20])
    plt.tight_layout()
    plt.show()

# Paths to the testing image folders
out_dist_folder = '/content/drive/MyDrive/out-dist-cxr'
in_dist_folder = '/content/drive/MyDrive/in_dist_pool'

# Display 5 random images from each testing folder
print("Out-of-Distribution Images (Non-CXR):")
display_random_images_from_directory(out_dist_folder)

print("In-Distribution Images (CXR):")
display_random_images_from_directory(in_dist_folder)

In [ ]:
# Load the base VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

In [ ]:
def extract_features_from_image(image_path, target_size=(224, 224)):
    img = Image.open(image_path).convert('RGB')  # Convert to RGB since VGG16 expects 3 channels
    img = img.resize(target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array * 255)
    features = model.predict(img_array)
    return features.flatten()

def extract_features_from_folder(folder_path):
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        return np.array([]), []
    image_paths = [os.path.join(folder_path, fname) for fname in os.listdir(folder_path)]
    features = []
    filenames = []
    for img_path in tqdm(image_paths):
        try:
            img_features = extract_features_from_image(img_path)
            features.append(img_features)
            filenames.append(os.path.basename(img_path))
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
    return np.array(features), filenames


def extract_features_from_generator(generator, model, num_samples):
    batch_size = generator.batch_size
    total_batches = (num_samples // batch_size) + (num_samples % batch_size > 0)
    all_features = []

    for _ in tqdm(range(total_batches)):
        batch_images, _ = next(generator)
        batch_features = model.predict(preprocess_input(batch_images * 255)).reshape(batch_images.shape[0], -1)
        all_features.append(batch_features)

    return np.vstack(all_features)

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
import numpy as np

def extract_features(samples):
    """Extract features from the samples using the model"""
    features = model.predict(samples, batch_size=32, verbose=1)
    return features

# Extracting features from the training images
train_features = extract_features(train_images)

# Reshape the features as needed (depends on the next steps of your workflow)
train_features = np.reshape(train_features, (train_features.shape[0], -1))


In [ ]:
train_features = extract_features_from_folder(data_dir)
train_features = np.reshape(train_features, (train_features.shape[0], -1))

print(train_features.shape)
print(len(train_features))

In [ ]:
# Extract features and filenames using extract_features_from_folder
train_features, train_filenames = extract_features_from_folder(data_dir)

# Now, train_features is just the NumPy array of extracted features
train_features = np.reshape(train_features, (train_features.shape[0], -1))

print(train_features.shape)
print(len(train_filenames))


In [ ]:
from keras.preprocessing.image import img_to_array

# Now, using extract_features_from_folder for the test images
in_dist_features, in_dist_filenames = extract_features_from_folder(in_dist_folder)
out_dist_features, out_dist_filenames = extract_features_from_folder(out_dist_folder)
print(in_dist_features.shape)
print(out_dist_features.shape)
print(len(in_dist_features))

In [ ]:
print(in_dist_features.shape)
print(out_dist_features.shape)
print(len(in_dist_features))
print(len(out_dist_features))

In [ ]:

# Replace these paths with your desired save locations
training_features_path = '/content/drive/My Drive/PadChest_supervised_training_features.npy'
in_dist_features_path = '/content/drive/My Drive/PadChest__supervised_in_dist_features.npy'
out_dist_features_path = '/content/drive/My Drive/PadChest_supervised_out_dist_features.npy'

# Ensure train_features is a numpy array
train_features = np.array(train_features)

# Check the shape of train_features
print("Shape of train_features:", train_features.shape)

# Now, try saving the features
training_features_path = "/content/drive/My Drive/PadChest_supervised_training_features.npy"  # replace with your desired path
np.save(training_features_path, train_features)

# Save the features
np.save(training_features_path, train_features)
np.save(in_dist_features_path, in_dist_features)
np.save(out_dist_features_path, out_dist_features)

print("Features saved successfully.")

In [ ]:
# Load the features
training_features = np.load(training_features_path)
in_dist_features = np.load(in_dist_features_path)
out_dist_features = np.load(out_dist_features_path)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Function to perform PCA and plot
def plot_pca(features, title, ax):
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(features)
    ax.scatter(pca_result[:, 0], pca_result[:, 1], alpha=0.5)
    ax.set_title(title)
    ax.set_xlabel('PCA Component 1')
    ax.set_ylabel('PCA Component 2')

# Create a figure with 3 subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot each dataset
plot_pca(training_features, 'Training Features', axes[0])
plot_pca(in_dist_features, 'In-Dist Features', axes[1])
plot_pca(out_dist_features, 'Out-Dist Features', axes[2])

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
from scipy.spatial import distance
from numpy.linalg import pinv

def compute_similarity(tr_features, tt_features, similarity_type='cosine'):
    """
    Compute similarities between training and testing features based on the specified type.

    Parameters:
    - tr_features (array): Training feature vectors.
    - tt_features (array): Testing feature vectors.
    - similarity_type (str): Type of similarity to compute ('cosine' or 'mahalanobis').

    Returns:
    - dict: Contains computed similarities and basic statistics.
    """

    def compute_cosine_similarity(tr_features_, tt_features_):
        centroid = np.mean(tr_features_, axis=0)
        return [1 - distance.cosine(feature, centroid) for feature in tt_features_]

    def compute_mahalanobis_similarity(tr_features_, tt_features_):
        covariance_matrix = np.cov(tr_features_, rowvar=False)
        covariance_matrix_inv = pinv(covariance_matrix)
        centroid = np.mean(tr_features_, axis=0)
        return [distance.mahalanobis(feature, centroid, covariance_matrix_inv) for feature in tt_features_]

    # Compute similarities
    if similarity_type == 'cosine':
        similarities = compute_cosine_similarity(tr_features, tt_features)
    elif similarity_type == 'mahalanobis':
        similarities = compute_mahalanobis_similarity(tr_features, tt_features)
    else:
        raise ValueError("Invalid similarity type. Choose 'cosine' or 'mahalanobis'.")

    # Compute statistics
    mean_similarity = np.mean(similarities)
    std_similarity = np.std(similarities)
    median_similarity = np.median(similarities)
    mad_similarity = np.median(np.abs(similarities - median_similarity))
    percentile_95 = np.percentile(similarities, 95)
    percentile_99 = np.percentile(similarities, 99)
    range_similarity = np.ptp(similarities)
    iqr_similarity = np.percentile(similarities, 75) - np.percentile(similarities, 25)

    return {
        'similarities': similarities,
        'mean': mean_similarity,
        'std': std_similarity,
        'median': median_similarity,
        'mad': mad_similarity,
        'percentile_95': percentile_95,
        'percentile_99': percentile_99,
        'range': range_similarity,
        'iqr': iqr_similarity
    }


In [ ]:
# Assuming train_features, test_in_dist_features, and test_out_dist_features are defined
cosine_train_similarities_VGG16_PadChest = compute_similarity(training_features, training_features, "cosine")
cosine_train_similarities_actaul_VGG16_PadChest = cosine_train_similarities_VGG16_PadChest['similarities']
cosine_mean_train_VGG16_PadChest = cosine_train_similarities_VGG16_PadChest["mean"]
cosine_std_train_VGG16_PadChest = cosine_train_similarities_VGG16_PadChest["std"]


# Computing the 3-sigma upper and lower control limits
cosine_3UCL_train_upper_VGG16_PadChest = cosine_mean_train_VGG16_PadChest + 3 * cosine_std_train_VGG16_PadChest
cosine_3LCL_train_lower_VGG16_PadChest = cosine_mean_train_VGG16_PadChest - 3 * cosine_std_train_VGG16_PadChest



cosine_in_dist_similarities_VGG16_PadChest = compute_similarity(training_features, in_dist_features, "cosine")
cosine_in_dist_similarities_VGG16_actual_PadChest = cosine_in_dist_similarities_VGG16_PadChest['similarities']

cosine_out_dist_similarities_VGG16_PadChest = compute_similarity(training_features, out_dist_features, "cosine")
cosine_out_dist_similarities_VGG16_actual_PadChest = cosine_out_dist_similarities_VGG16_PadChest['similarities']

cosine_in_dist_similarities_VGG16_mean_PadChest = cosine_in_dist_similarities_VGG16_PadChest["mean"]
cosine_in_dist_similarities_VGG16_std_PadChest = cosine_in_dist_similarities_VGG16_PadChest["std"]

cosine_out_dist_similarities_VGG16_mean_PadChest = cosine_out_dist_similarities_VGG16_PadChest["mean"]
cosine_out_dist_similarities_VGG16_std_PadChest = cosine_out_dist_similarities_VGG16_PadChest["std"]

print("Cosine - Mean train: {} | STD train: {}".format(cosine_mean_train_VGG16_PadChest, cosine_std_train_VGG16_PadChest))
print("Cosine - MEAN in-dist: {} | MEAN out-dist: {}".format(cosine_in_dist_similarities_VGG16_mean_PadChest, cosine_in_dist_similarities_VGG16_std_PadChest))
print("Cosine - STD out-dist: {} | STD out-dist: {}".format(cosine_out_dist_similarities_VGG16_mean_PadChest, cosine_out_dist_similarities_VGG16_std_PadChest))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create histograms
plt.figure(figsize=(18, 6))

# Training Features Histogram
plt.subplot(1, 3, 1)
plt.hist(cosine_train_similarities_actaul_VGG16_PadChest, bins=50, color='blue', alpha=0.7)
plt.title('Train Features Cosine Similarity')
plt.xlabel('Cosine Similarity')
plt.ylabel('Frequency')

# In-Dist Features Histogram
plt.subplot(1, 3, 2)
plt.hist(cosine_in_dist_similarities_VGG16_actual_PadChest, bins=50, color='green', alpha=0.7)
plt.title('In-Dist Features Cosine Similarity')
plt.xlabel('Cosine Similarity')
plt.ylabel('Frequency')

# Out-Dist Features Histogram
plt.subplot(1, 3, 3)
plt.hist(cosine_out_dist_similarities_VGG16_actual_PadChest, bins=50, color='red', alpha=0.7)
plt.title('Out-Dist Features Cosine Similarity')
plt.xlabel('Cosine Similarity')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Randomly pick 50 points from in-distribution and out-of-distribution cosine similarities
in_dist_samples = np.random.choice(cosine_in_dist_similarities_VGG16_actual_PadChest, 50, replace=False)
out_dist_samples = np.random.choice(cosine_out_dist_similarities_VGG16_actual_PadChest, 50, replace=False)

# Combine and shuffle these 100 points
combined_samples = np.concatenate((in_dist_samples, out_dist_samples))
# Also, create a label array to track InD (0) and OOD (1) points
labels = np.array([0]*50 + [1]*50)
# Shuffle both arrays in unison
shuffled_indices = np.random.permutation(np.arange(100))
combined_samples = combined_samples[shuffled_indices]
labels = labels[shuffled_indices]
#np.random.shuffle(combined_samples)
print(combined_samples)

# Identify points as in or out of distribution based on control limits
ood_predictions = [1 if val < cosine_3LCL_train_lower_VGG16_PadChest else 0 for val in combined_samples]

In [ ]:
import matplotlib.pyplot as plt

# Assuming the following are defined:
# combined_samples - contains the mixed in-distribution and out-of-distribution points
# ood_predictions - flags for points detected as out-of-distribution based on control limits
# labels - actual labels indicating whether a point is in-distribution (0) or out-of-distribution (1)
# cosine_3UCL_train_upper, cosine_3LCL_train_lower - control limits
# cosine_mean_train - mean cosine similarity for the training set

# Plot the SPC chart
fig, ax = plt.subplots(figsize=(15, 6))
ax.plot(combined_samples, marker='o', linestyle='-', color='grey', zorder=1)

# Plot control limits and mean
ax.axhline(np.clip(cosine_3UCL_train_upper_VGG16_PadChest, a_min=0.0, a_max=1.0), color='k', linestyle='--', label='Control Limit (UCL & LCL)')
ax.axhline(np.clip(cosine_3LCL_train_lower_VGG16_PadChest, a_min=0.0, a_max=1.0), color='k', linestyle='--')
ax.axhline(cosine_mean_train_VGG16_PadChest, color='k', linestyle='-', label='Mean')

# Highlight points detected as out-of-distribution with black stars
for i, val in enumerate(combined_samples):
    if ood_predictions[i] == 1:
        ax.scatter(i, val, color='black', marker='o', s=60, zorder=2)

# Circle actual out-of-distribution points with blue circles
for i, val in enumerate(combined_samples):
    if labels[i] == 1:
        ax.scatter(i, val, marker='s', facecolors='none', edgecolors='blue', s=120, zorder=3)

# Set chart properties
ax.set_facecolor('white')
ax.grid(True, which='both', linestyle='--', linewidth=0.5)

# Update the legend
legend_elements = [plt.Line2D([0], [0], color='k', linestyle='--', label='Control Limit (UCL & LCL)'),
                   plt.Line2D([0], [0], color='k', linestyle='-', label='Mean'),
                   plt.Line2D([0], [0], marker='o', color='w', label='Detected OOD', markerfacecolor='black', markersize=10),
                   plt.Line2D([0], [0], marker='s', color='w', label='Actual OOD', markerfacecolor='none', markeredgecolor='blue', markersize=10)]
ax.legend(handles=legend_elements)

# Set chart properties (adding the axis labels here)
ax.set_xlabel('Image Samples')  # X-axis label
ax.set_ylabel('OOD Metric')    # Y-axis label
ax.set_facecolor('white')
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
# Show the plot
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import resample

# Combine the cosine similarity scores and their corresponding labels
combined_scores = np.concatenate([cosine_in_dist_similarities_VGG16_actual_PadChest, cosine_out_dist_similarities_VGG16_actual_PadChest])
# Label: 1 for out-of-distribution, 0 for in-distribution
combined_labels = np.concatenate([np.zeros(len(cosine_in_dist_similarities_VGG16_actual_PadChest)), np.ones(len(cosine_out_dist_similarities_VGG16_actual_PadChest))])

# Apply the rule to flag points
ood_predictions = np.array([1 if val < cosine_3LCL_train_lower_VGG16_PadChest else 0 for val in combined_scores])

# Function to compute metrics
def compute_metrics(y_true, y_pred):
    sensitivity = recall_score(y_true, y_pred)  # Same as recall
    specificity = recall_score(y_true, y_pred, pos_label=0)
    accuracy = accuracy_score(y_true, y_pred)
    return sensitivity, specificity, accuracy

# Compute initial metrics
initial_sensitivity, initial_specificity, initial_accuracy = compute_metrics(combined_labels, ood_predictions)

# Bootstrapping for confidence intervals
n_iterations = 500
n_size = int(len(combined_labels) * 0.5)  # 50% sample size
bootstrapped_metrics = []

for _ in range(n_iterations):
    # Prepare bootstrap sample
    indices = resample(np.arange(len(combined_labels)), n_samples=n_size)
    boot_labels = combined_labels[indices]
    boot_predictions = ood_predictions[indices]

    # Compute metrics
    metrics = compute_metrics(boot_labels, boot_predictions)
    bootstrapped_metrics.append(metrics)

# Convert to numpy array for easy percentile computation
bootstrapped_metrics = np.array(bootstrapped_metrics)

# Compute 95% confidence intervals
lower_p = 2.5
upper_p = 97.5
sensitivity_conf = np.percentile(bootstrapped_metrics[:, 0], [lower_p, upper_p])
specificity_conf = np.percentile(bootstrapped_metrics[:, 1], [lower_p, upper_p])
accuracy_conf = np.percentile(bootstrapped_metrics[:, 2], [lower_p, upper_p])

# Compute 95% confidence intervals
lower_p = 2.5
upper_p = 97.5
sensitivity_conf = np.percentile(bootstrapped_metrics[:, 0], [lower_p, upper_p])
specificity_conf = np.percentile(bootstrapped_metrics[:, 1], [lower_p, upper_p])
accuracy_conf = np.percentile(bootstrapped_metrics[:, 2], [lower_p, upper_p])

print("Supervised features (initial_sensitivity)", initial_sensitivity)
print("Supervised features (sensitivity_conf)", sensitivity_conf)

print("Supervised features (initial_specificity)", initial_specificity)
print("Supervised features (specificity_conf)", specificity_conf)

print("Supervised features (initial_accuracy)", initial_accuracy)
print("Supervised features (accuracy_conf)", accuracy_conf)

### **Pedatric CXR Test: **
Now, we use the same training set but the test set will be pedatric!

In [ ]:
# Function to display random images from a directory
def display_random_images_from_directory(directory, num_images=5):
    image_names = [name for name in os.listdir(directory) if name.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]
    random_images = random.sample(image_names, min(num_images, len(image_names)))

    plt.figure(figsize=(10, 10))
    for i, image_name in enumerate(random_images):
        img_path = os.path.join(directory, image_name)
        img = Image.open(img_path).convert('L')
        img = img.resize((256, 256), Image.ANTIALIAS)
        plt.subplot(1, 5, i + 1)
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        plt.title(image_name[:20])
    plt.tight_layout()
    plt.show()

# Paths to the testing image folders
out_dist_folder_pedatric = '/content/drive/MyDrive/Pedatric_CXR'

# Display 5 random images from each testing folder
print("Out-of-Distribution Images (CXR-Pedatric):")
display_random_images_from_directory(out_dist_folder_pedatric)


In [ ]:
# Now, using extract_features_from_folder for the test images
in_dist_features, in_dist_filenames = extract_features_from_folder(in_dist_folder)
out_dist_features, out_dist_filenames = extract_features_from_folder(out_dist_folder)
out_dist_features_pedatric, out_dist_filenames_pedatric = extract_features_from_folder(out_dist_folder_pedatric)
print(in_dist_features.shape)
print(out_dist_features.shape)
print(len(in_dist_features))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Function to perform PCA and plot
def plot_pca(features, title, ax):
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(features)
    ax.scatter(pca_result[:, 0], pca_result[:, 1], alpha=0.5)
    ax.set_title(title)
    ax.set_xlabel('PCA Component 1')
    ax.set_ylabel('PCA Component 2')

# Create a figure with 3 subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot each dataset
plot_pca(training_features, 'Training Features', axes[0])
plot_pca(in_dist_features, 'In-Dist Features', axes[1])
plot_pca(out_dist_features_pedatric, 'Out-Dist Pedatric Features', axes[2])

plt.tight_layout()
plt.show()

In [ ]:


cosine_out_dist_similarities_VGG16_Pedatric = compute_similarity(training_features, out_dist_features_pedatric, "cosine")
cosine_out_dist_similarities_VGG16_actual_Pedatric = cosine_out_dist_similarities_VGG16_Pedatric['similarities']

cosine_out_dist_similarities_VGG16_mean_Pedatric = cosine_out_dist_similarities_VGG16_Pedatric["mean"]
cosine_out_dist_similarities_VGG16_std_Pedatric = cosine_out_dist_similarities_VGG16_Pedatric["std"]

print("Cosine - Mean train: {} | STD train: {}".format(cosine_mean_train_VGG16_PadChest, cosine_std_train_VGG16_PadChest))
print("Cosine - MEAN in-dist: {} | MEAN out-dist: {}".format(cosine_in_dist_similarities_VGG16_mean_PadChest, cosine_in_dist_similarities_VGG16_std_PadChest))
print("Cosine - STD out-dist Pedatric: {} | STD out-dist Pedatric: {}".format(cosine_out_dist_similarities_VGG16_mean_Pedatric, cosine_out_dist_similarities_VGG16_std_Pedatric))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create histograms
plt.figure(figsize=(18, 6))

# Training Features Histogram
plt.subplot(1, 3, 1)
plt.hist(cosine_train_similarities_actaul_VGG16_PadChest, bins=50, color='blue', alpha=0.7)
plt.title('Train Features Cosine Similarity')
plt.xlabel('Cosine Similarity')
plt.ylabel('Frequency')

# In-Dist Features Histogram
plt.subplot(1, 3, 2)
plt.hist(cosine_in_dist_similarities_VGG16_actual_PadChest, bins=50, color='green', alpha=0.7)
plt.title('In-Dist Features Cosine Similarity')
plt.xlabel('Cosine Similarity')
plt.ylabel('Frequency')

# Out-Dist Features Histogram
plt.subplot(1, 3, 3)
plt.hist(cosine_out_dist_similarities_VGG16_actual_Pedatric, bins=50, color='red', alpha=0.7)
plt.title('Out-Dist Pedatric Features Cosine Similarity')
plt.xlabel('Cosine Similarity')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Randomly pick 50 points from in-distribution and out-of-distribution cosine similarities
in_dist_samples = np.random.choice(cosine_in_dist_similarities_VGG16_actual_PadChest, 50, replace=False)
out_dist_samples_pedatric = np.random.choice(cosine_out_dist_similarities_VGG16_actual_Pedatric, 50, replace=False)

# Combine and shuffle these 100 points
combined_samples = np.concatenate((in_dist_samples, out_dist_samples_pedatric))
# Also, create a label array to track InD (0) and OOD (1) points
labels = np.array([0]*50 + [1]*50)
# Shuffle both arrays in unison
shuffled_indices = np.random.permutation(np.arange(100))
combined_samples = combined_samples[shuffled_indices]
labels = labels[shuffled_indices]
#np.random.shuffle(combined_samples)
print(combined_samples)


# Computing the 3-sigma upper and lower control limits
cosine_3UCL_train_upper_VGG16_PadChest_2Sigma = cosine_mean_train_VGG16_PadChest + 1 * cosine_std_train_VGG16_PadChest
cosine_3LCL_train_lower_VGG16_PadChest__2Sigma = cosine_mean_train_VGG16_PadChest - 1 * cosine_std_train_VGG16_PadChest


# Identify points as in or out of distribution based on control limits
ood_predictions = [1 if val < cosine_3LCL_train_lower_VGG16_PadChest__2Sigma else 0 for val in combined_samples]

In [ ]:
import matplotlib.pyplot as plt

# Assuming the following are defined:
# combined_samples - contains the mixed in-distribution and out-of-distribution points
# ood_predictions - flags for points detected as out-of-distribution based on control limits
# labels - actual labels indicating whether a point is in-distribution (0) or out-of-distribution (1)
# cosine_3UCL_train_upper, cosine_3LCL_train_lower - control limits
# cosine_mean_train - mean cosine similarity for the training set

# Plot the SPC chart
fig, ax = plt.subplots(figsize=(15, 6))
ax.plot(combined_samples, marker='o', linestyle='-', color='grey', zorder=1)

# Plot control limits and mean
ax.axhline(np.clip(cosine_3UCL_train_upper_VGG16_PadChest_2Sigma, a_min=0.0, a_max=1.0), color='k', linestyle='--', label='Control Limit (UCL & LCL)')
ax.axhline(np.clip(cosine_3LCL_train_lower_VGG16_PadChest__2Sigma, a_min=0.0, a_max=1.0), color='k', linestyle='--')
ax.axhline(cosine_mean_train_VGG16_PadChest, color='k', linestyle='-', label='Mean')

# Highlight points detected as out-of-distribution with black stars
for i, val in enumerate(combined_samples):
    if ood_predictions[i] == 1:
        ax.scatter(i, val, color='black', marker='o', s=60, zorder=2)

# Circle actual out-of-distribution points with blue circles
for i, val in enumerate(combined_samples):
    if labels[i] == 1:
        ax.scatter(i, val, marker='s', facecolors='none', edgecolors='blue', s=120, zorder=3)

# Set chart properties
ax.set_facecolor('white')
ax.grid(True, which='both', linestyle='--', linewidth=0.5)

# Update the legend
legend_elements = [plt.Line2D([0], [0], color='k', linestyle='--', label='Control Limit (UCL & LCL)'),
                   plt.Line2D([0], [0], color='k', linestyle='-', label='Mean'),
                   plt.Line2D([0], [0], marker='o', color='w', label='Detected OOD', markerfacecolor='black', markersize=10),
                   plt.Line2D([0], [0], marker='s', color='w', label='Actual OOD', markerfacecolor='none', markeredgecolor='blue', markersize=10)]
ax.legend(handles=legend_elements)

# Set chart properties (adding the axis labels here)
ax.set_xlabel('Image Samples')  # X-axis label
ax.set_ylabel('OOD Metric')    # Y-axis label
ax.set_facecolor('white')
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
# Show the plot
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import resample

# Combine the cosine similarity scores and their corresponding labels
combined_scores = np.concatenate([cosine_in_dist_similarities_VGG16_actual_PadChest, cosine_out_dist_similarities_VGG16_actual_Pedatric])
# Label: 1 for out-of-distribution, 0 for in-distribution
combined_labels = np.concatenate([np.zeros(len(cosine_in_dist_similarities_VGG16_actual_PadChest)), np.ones(len(cosine_out_dist_similarities_VGG16_actual_Pedatric))])

# Apply the rule to flag points
ood_predictions = np.array([1 if val < cosine_3LCL_train_lower_VGG16_PadChest__2Sigma else 0 for val in combined_scores])

# Function to compute metrics
def compute_metrics(y_true, y_pred):
    sensitivity = recall_score(y_true, y_pred)  # Same as recall
    specificity = recall_score(y_true, y_pred, pos_label=0)
    accuracy = accuracy_score(y_true, y_pred)
    return sensitivity, specificity, accuracy

# Compute initial metrics
initial_sensitivity, initial_specificity, initial_accuracy = compute_metrics(combined_labels, ood_predictions)

# Bootstrapping for confidence intervals
n_iterations = 500
n_size = int(len(combined_labels) * 0.5)  # 50% sample size
bootstrapped_metrics = []

for _ in range(n_iterations):
    # Prepare bootstrap sample
    indices = resample(np.arange(len(combined_labels)), n_samples=n_size)
    boot_labels = combined_labels[indices]
    boot_predictions = ood_predictions[indices]

    # Compute metrics
    metrics = compute_metrics(boot_labels, boot_predictions)
    bootstrapped_metrics.append(metrics)

# Convert to numpy array for easy percentile computation
bootstrapped_metrics = np.array(bootstrapped_metrics)

# Compute 95% confidence intervals
lower_p = 2.5
upper_p = 97.5
sensitivity_conf = np.percentile(bootstrapped_metrics[:, 0], [lower_p, upper_p])
specificity_conf = np.percentile(bootstrapped_metrics[:, 1], [lower_p, upper_p])
accuracy_conf = np.percentile(bootstrapped_metrics[:, 2], [lower_p, upper_p])

# Compute 95% confidence intervals
lower_p = 2.5
upper_p = 97.5
sensitivity_conf = np.percentile(bootstrapped_metrics[:, 0], [lower_p, upper_p])
specificity_conf = np.percentile(bootstrapped_metrics[:, 1], [lower_p, upper_p])
accuracy_conf = np.percentile(bootstrapped_metrics[:, 2], [lower_p, upper_p])

print("Pedatric features (initial_sensitivity)", initial_sensitivity)
print("Pedatric features (sensitivity_conf)", sensitivity_conf)

print("Pedatric features (initial_specificity)", initial_specificity)
print("Pedatric features (specificity_conf)", specificity_conf)

print("Pedatric features (initial_accuracy)", initial_accuracy)
print("Pedatric features (accuracy_conf)", accuracy_conf)

In [ ]:
# Now, I want to 60 images and use these samples to create the follwoing charts:
# Three-sigma chart, Bernouuli CUSUM, CUSUM


# Take 30 random samples without replacement from each
in_dist_samples = np.random.choice(cosine_in_dist_similarities_VGG16_actual_PadChest, 30, replace=False)
out_dist_samples_pedatric = np.random.choice(cosine_out_dist_similarities_VGG16_actual_Pedatric, 30, replace=False)

# Combine the two sets of samples into one vector
combined_in_out_samples = np.concatenate((in_dist_samples, out_dist_samples_pedatric))
print(combined_samples)

In [ ]:
shift_start_day = 30  # The point at which the shift occurs
mu0_in_control = np.mean(combined_in_out_samples[:shift_start_day])  # Mean of the in-control data
std0_in_control = np.std(combined_in_out_samples[:shift_start_day])  # Standard deviation of the in-control data
print(mu0_in_control)
print(std0_in_control)

In [ ]:
# Select CUSUM paramters

# Initialize lists to store results
FalsePos = []
TruePos = []
AvgDD = []  # Average Detection Delay
DetectionTimes = []

# Define the range of k values as a fraction of in_std
k_values = [0.1, 0.25, 0.5, 0.75, 1.0]

# Specify the control parameters and the threshold
pre_change_days = 30  # Number of days the process is in-control
total_days = 60  # Total number of days in the dataset
control_limit = 4  # Multiplier for control limit
delta = 1  # Change magnitude in terms of standard deviations

# Split your data into in-control and out-of-control periods
in_control_sp = combined_in_out_samples[:pre_change_days]
out_control_sp = combined_in_out_samples[pre_change_days:total_days]

# Compute the mean and standard deviation for in-control and out-of-control periods
mu_in = np.mean(in_control_sp)
mu_out = np.mean(out_control_sp)
in_std = np.std(in_control_sp)

#k = (delta * in_std) / 2
print(len(combined_in_out_samples))
print(mu_in)
print(mu_out)

In [ ]:
# This function creates a daily pool of in-dist and out-dist test set based on the decided percentage of shift

# Define the number of days and distances per day
total_days = 60
distances_per_day = 100

# Create an empty list to store daily data
all_data = []

# Function to select data for each day
def select_daily_data(in_dist_data, out_dist_data, out_dist_percent, day_data_count):
    out_dist_count = int(day_data_count * out_dist_percent / 100)
    in_dist_count = day_data_count - out_dist_count

    # Randomly select out-of-distribution and in-distribution data
    daily_out_dist_data = np.random.choice(out_dist_data, out_dist_count, replace=False)
    daily_in_dist_data = np.random.choice(in_dist_data, in_dist_count, replace=False)

    # Combine and return the daily data
    return np.concatenate([daily_out_dist_data, daily_in_dist_data])

 # Simulate data for each day. This case is a small shift
for day in range(1, total_days + 1):
    if day <= 30:
        percent_out_dist = 2  # First month: pre shift
    elif day <= 60:
        percent_out_dist = 4  # First Shift: 4% OOD
    # elif day <= 70:
    #    percent_out_dist = 7 # Second Shift: 7% OOD
    # elif day <= 100:
    #    percent_out_dist = 12  # Third Shift: 12% OOD
    #elif day <= 150:
    #    percent_out_dist = 10 # Third month: 28% OOD
    #else:
    #    percent_out_dist = 5 # Fourth month: 7% OOD

    daily_data = select_daily_data(
        cosine_in_dist_similarities_VGG16_actual_PadChest,
        cosine_out_dist_similarities_VGG16_actual_Pedatric,
        percent_out_dist,
        distances_per_day
    )
    all_data.append(daily_data)


# Combine all daily data for CUSUM analysis
avg_cusum_data = np.concatenate(all_data)
print(len(avg_cusum_data))

# This output can be used by 2-sigma and 3-sigma chart, as we are only using one value with them.

In [ ]:
# The previous function uses single images for 2-sigma and 3-sigma charts
# For CUSUM, we will use the average per day

# Calculate the daily averages and the percentage of out-of-distribution data
daily_averages = [np.mean(day) for day in all_data]
percentages_out_dist = []

for day in range(1, total_days + 1):
      if day <= 30:
        percent_out_dist = 2  # First month: 3% OOD
      elif day <= 60:
        percent_out_dist = 4  # Second month: 4% OOD
      #elif day <= 70:
      #  percent_out_dist = 7 # Third month: 7% OOD
      #elif day <= 100:
      #  percent_out_dist = 12  # Third month: 12% OOD
      #elif day <= 150:
      #  percent_out_dist = 10 # Third montn: 28% OOD
      #else:
      #  percent_out_dist = 5 # Fourth month: 7% OOD
      percentages_out_dist.append(percent_out_dist)

# Plotting
plt.figure(figsize=(15, 6))
sc = plt.scatter(range(1, total_days + 1), daily_averages, c=percentages_out_dist, cmap='coolwarm')
plt.colorbar(sc, label='Percentage of Out-of-Distribution Data (%)')
plt.title('Daily Average Cosine Distance with Out-of-Distribution Data Percentage')
plt.xlabel('Day')
plt.ylabel('Average Cosine Distance')
plt.grid(True)
plt.show()


# Calculate the daily averages
daily_averages = [np.mean(day) for day in all_data]
CUSUM_data_average_day = np.array(daily_averages)
print(len(CUSUM_data_average_day))

# Plotting
plt.figure(figsize=(15, 6))
plt.plot(range(1, total_days + 1), daily_averages, marker='o', linestyle='-', color='b')
plt.title('Daily Average Cosine Distance Over Time')
plt.xlabel('Day')
plt.ylabel('Average Cosine Distance')
plt.grid(True)
plt.show()

# Now daily_averages can be used for further analysis like CUSUM
#daily_averages[:60]  # Displaying the first 10 days as a sample
#print(len(daily_averages))
#print(np.mean(daily_averages[1:15]) )
#print(np.mean(daily_averages[16:30]) )


# Save the cosine similarity for each batch for each day.
np.savetxt("/content/drive/My Drive/daily_averages_cosine_CUSUM_VGG16.csv", daily_averages, delimiter=",", fmt='%s')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming the function `select_daily_data_with_labels` is already defined as earlier
# Let's redefine it here for completeness
def select_daily_data_with_labels(in_dist_data, out_dist_data, out_dist_percent, day_data_count):
    out_dist_count = int(day_data_count * out_dist_percent / 100)
    in_dist_count = day_data_count - out_dist_count

    daily_out_dist_data = np.random.choice(out_dist_data, out_dist_count, replace=False)
    daily_in_dist_data = np.random.choice(in_dist_data, in_dist_count, replace=False)

    daily_out_dist_labels = ['out-dist'] * out_dist_count
    daily_in_dist_labels = ['in-dist'] * in_dist_count

    combined_data = np.concatenate((daily_in_dist_data, daily_out_dist_data))
    combined_labels = np.concatenate((daily_in_dist_labels, daily_out_dist_labels))

    combined = list(zip(combined_data, combined_labels))
    np.random.shuffle(combined)

    shuffled_data, shuffled_labels = zip(*combined)
    return list(shuffled_data), list(shuffled_labels)

# Assuming cosine_in_dist_similarities and cosine_out_dist_similarities are already defined
# as arrays of cosine similarity values for in-distribution and out-of-distribution data

# Now we select the data for the four days
day_data = {}
day_labels = {}
# percentages_out_dist = [3, 4, 7, 12]  # Percentage for each period
percentages_out_dist = [5, 12]  # Percentage for each period


# Select one day before day 30
day_data[1], day_labels[1] = select_daily_data_with_labels(
    cosine_in_dist_similarities_VGG16_actual_PadChest,
    cosine_out_dist_similarities_VGG16_actual_Pedatric,
    percentages_out_dist[0],  # 3% out-dist
    100
)

# Select one day between day 31 and day 60
day_data[2], day_labels[2] = select_daily_data_with_labels(
    cosine_in_dist_similarities_VGG16_actual_PadChest,
    cosine_out_dist_similarities_VGG16_actual_Pedatric,
    percentages_out_dist[1],  # 4% out-dist
    100
)

# Select one day between day 41 and day 69
# day_data[3], day_labels[3] = select_daily_data_with_labels(
#    cosine_in_dist_similarities,
#    cosine_out_dist_similarities,
#    percentages_out_dist[2],  # 7% out-dist
#    60
#)

# Select one day between day 71 and day 99
#day_data[4], day_labels[4] = select_daily_data_with_labels(
#    cosine_in_dist_similarities,
#    cosine_out_dist_similarities,
#    percentages_out_dist[3],  # 12% out-dist
#    60
#)

# Now day_data contains the cosine similarities for the selected days
# and day_labels contains the corresponding labels

# We can now print the cosine similarities for each selected day
for day in day_data:
    print(f"Day {day} Data:")
    print(day_data[day])
    print(f"Day {day} Labels:")
    print(day_labels[day])
    print("\n")  # Print a newline for better readability between days

#day_labels = {
#    1: ['in-dist'] * 58 + ['out-dist'] * 2,
#    2: ['in-dist'] * 57 + ['out-dist'] * 3,
#    #3: ['in-dist'] * 56 + ['out-dist'] * 4,
    #4: ['in-dist'] * 53 + ['out-dist'] * 7
#}


# Percentages of out-of-distribution data for each day
percentages_out_dist = [5, 12]  # Extend this list as needed for more days

day_labels = {}

for day, out_dist_percentage in enumerate(percentages_out_dist, start=1):
    total_data_count = 100  # Assuming each day has 100 data points
    out_dist_count = int(total_data_count * out_dist_percentage / 100)
    in_dist_count = total_data_count - out_dist_count

    day_labels[day] = ['in-dist'] * in_dist_count + ['out-dist'] * out_dist_count

# Now day_labels contains dynamically assigned labels based on percentages

# Example: Print labels for each day
for day, labels in day_labels.items():
    print(f"Day {day} Labels:")
    print(labels)
    print("\n")

# Now we'll count the number of in-dist and out-dist points for each day.
in_dist_count = {}
out_dist_count = {}

for day, labels in day_labels.items():
    in_dist_count[day] = labels.count('in-dist')
    out_dist_count[day] = labels.count('out-dist')

in_dist_count, out_dist_count

In [ ]:
 sample_days = [15, 35, 65, 95]
sample_days = [18, 47]


# Calculate the mean and standard deviation for in-distribution data
mean_in_dist_day_bacth = np.mean(cosine_in_dist_similarities_VGG16_actual_PadChest)
std_in_dist_day_batch = np.std(cosine_in_dist_similarities_VGG16_actual_PadChest)

# Define the control limits using the three-sigma rule for in-distribution data
upper_control_limit = mean_in_dist_day_bacth + 3 * std_in_dist_day_batch
lower_control_limit = mean_in_dist_day_bacth - 3 * std_in_dist_day_batch


# Define the function to select daily data with labels
def select_daily_data_with_labels(in_dist_data, out_dist_data, out_dist_percent, day_data_count):
    out_dist_count = int(day_data_count * out_dist_percent / 100)
    in_dist_count = day_data_count - out_dist_count

    daily_out_dist_data = np.random.choice(out_dist_data, out_dist_count, replace=False)
    daily_in_dist_data = np.random.choice(in_dist_data, in_dist_count, replace=False)

    daily_out_dist_labels = ['out-dist'] * out_dist_count
    daily_in_dist_labels = ['in-dist'] * in_dist_count

    combined_data = np.concatenate((daily_in_dist_data, daily_out_dist_data))
    combined_labels = np.concatenate((daily_in_dist_labels, daily_out_dist_labels))

    combined = list(zip(combined_data, combined_labels))
    np.random.shuffle(combined)

    shuffled_data, shuffled_labels = zip(*combined)
    return list(shuffled_data), list(shuffled_labels)

# Now we select the data for the four days with the specified out-of-distribution percentages
# percentages_out_dist = [3, 4, 7, 12]  # Percentage for each period
percentages_out_dist = [5,12]  # Percentage for each period
day_data = {}
day_labels = {}

for i, percent_out_dist in enumerate(percentages_out_dist, start=1):
    day_data[i], day_labels[i] = select_daily_data_with_labels(
        cosine_in_dist_similarities_VGG16_actual_PadChest,
        cosine_out_dist_similarities_VGG16_actual_Pedatric,
        percent_out_dist,
        100
    )

# Let's create a function to plot SPC chart for given day
def plot_spc_chart(day_data, day_labels, upper_control_limit, lower_control_limit, day_number):
    # Plot the SPC chart
    fig, ax = plt.subplots(figsize=(15, 6))
    ax.plot(day_data, marker='o', linestyle='-', color='k', label=f' {day_number} Data')  # Black color for plot

    # Plot control limits and mean
    ax.axhline(upper_control_limit, color='k', linestyle='--', label='Upper Control Limit (UCL)')  # Black color for UCL
    ax.axhline(lower_control_limit, color='k', linestyle='--', label='Lower Control Limit (LCL)')  # Black color for LCL
    ax.axhline(np.mean(daily_averages[1:20]), color='k', linestyle='-', label='Mean')  # Black color for mean


    # Highlight out-of-distribution points
    for i, (val, label) in enumerate(zip(day_data, day_labels)):
        if label == 'out-dist':
            ax.scatter(i, val, color='darkgrey', marker='*', s=150)  # Red circles for out-of-control points
    # Highlight out-of-distribution points and draw a circle around them as known ground truth
    for i, (val, label) in enumerate(zip(day_data, day_labels)):
        if label == 'out-dist':
            #ax.scatter(i, val, color='grey', marker='o', s=250)  # Red circles for known out-of-control points
            ax.scatter(i, val, facecolors='none', edgecolors='grey', marker='o', s=250)  # Circle around the point





    ax.set_facecolor('white')  # White background
    ax.set_title(f'SPC Chart for {day_number}')
    ax.set_xlabel('Image Sequence')
    ax.set_ylabel('Cosine Similarity')
    ax.legend()
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)  # Lighter grid lines for better visibility


    plt.show()



# Now we plot the SPC charts with "Day 15", "Day 35", "Day 65", and "Day 95" as titles
for day, day_number in enumerate(sample_days, start=1):
  day_title = f'Day {day_number}'  # Create a title string for the chart
  plot_spc_chart(day_data[day], day_labels[day], upper_control_limit, lower_control_limit, day_title)


In [ ]:
# Three sigma chart for the average vaue
# Define the control limits using the three-sigma rule
upper_control_limit = np.mean(daily_averages[:30]) + 3 * np.std(daily_averages[:30])
lower_control_limit = np.mean(daily_averages[:30]) - 3 * np.std(daily_averages[:30])

print(np.mean(daily_averages[1:30]))
print(np.mean(daily_averages[31:60]))

shift_start_day = 31
#print(upper_control_limit)
#print(lower_control_limit)

# Create Bernoulli CUSUM vectors
average_binary_3sigma_vector = create_control_vector(daily_averages, upper_control_limit, lower_control_limit)
average_ground_truth_vector = create_shift_vector(len(daily_averages), shift_start_day)


# Plot the SPC chart
fig, ax = plt.subplots(figsize=(15, 6))
ax.plot(daily_averages, marker='o', linestyle='-', color='k', label='Daily Averages')  # Black color for plot

# Plot control limits and mean
ax.axhline(upper_control_limit, color='k', linestyle='--', label='Upper Control Limit (UCL)')  # Black color for UCL
ax.axhline(lower_control_limit, color='k', linestyle='--', label='Lower Control Limit (LCL)')  # Black color for LCL
ax.axhline(np.mean(daily_averages[1:30]), color='k', linestyle='-', label='Mean')  # Black color for mean

# Highlight points outside of control limits with grey circle around them
for i, val in enumerate(daily_averages):
    if val > upper_control_limit or val < lower_control_limit:
        ax.scatter(i, val, color='darkgrey', marker='*', s=150)  # Dark grey stars for out-of-control points
        #ax.scatter(i, val, facecolors='none', edgecolors='grey', marker='o', s=250)  # Grey circle around out-of-control points

# Circle all points after the shift point
for i in range(30, len(daily_averages)):
    ax.scatter(i, daily_averages[i], facecolors='none', edgecolors='grey', marker='o', s=250)  # Grey circle around points after shift



# Indicate the first shift point
ax.axvline(x=30, color='purple', linestyle='--', label='Low Shift (2-4%)')  # Purple line for shift start
# Indicate the second shift point
#ax.axvline(x=40, color='purple', linestyle='--', label='Second Shift (moderate)')  # Purple line for shift start
# Indicate the third shift point
#ax.axvline(x=70, color='purple', linestyle='--', label='Third Shift (high)')  # Purple line for shift start

ax.set_facecolor('white')  # White background
ax.set_xlabel('Time (Day)')
ax.set_ylabel('Average Cosine Similarity ')
ax.legend()
ax.grid(True, which='both', linestyle='--', linewidth=0.5)  # Lighter grid lines for better visibility

# Show the plot
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,classification_report, f1_score

conf_matrix = confusion_matrix(average_ground_truth_vector, average_binary_3sigma_vector)

# Calculate True Positive (TP), False Positive (FP), True Negative (TN), and False Negative (FN)
TP, FP, FN, TN = conf_matrix.ravel()

# Calculate accuracy, sensitivity (recall), and specificity
accuracy = accuracy_score(average_ground_truth_vector, average_binary_3sigma_vector)
sensitivity = TP / (TP + FN)  # Recall
specificity = TN / (TN + FP)

# Print the metrics
print("Confusion Matrix:")
print(conf_matrix)

print("\nAccuracy: {:.2f}%".format(accuracy * 100))
print("Sensitivity (Recall): {:.2f}%".format(sensitivity * 100))
print("Specificity: {:.2f}%".format(specificity * 100))



In [ ]:
# Define CUSUM function
def CUSUM(x, mu0, k, h):
    S_hi = [0]
    S_lo = [0]
    for i in range(len(x)):
        S_hi.append(max(0, S_hi[i] + (x[i] - mu0 - k)))
        S_lo.append(min(0, S_lo[i] + (x[i] - mu0 + k)))

    S_hi = np.array(S_hi[1:])
    S_lo = np.array(S_lo[1:])

    signal_hi = np.where(S_hi > h)[0]
    signal_lo = np.where(S_lo < -h)[0]
    signal = np.unique(np.concatenate((signal_hi, signal_lo)))

    return signal, S_hi, S_lo

In [ ]:
# Define the range of k values as a fraction of in_std
k_values = [0.1, 0.25, 0.5, 0.75, 1.0]
# Specify the control parameters and the threshold
pre_change_days = 30  # Number of days the process is in-control
total_days = 60  # Total number of days in the dataset
control_limit = 4  # Multiplier for control limit
delta = 1  # Change magnitude in terms of standard deviations

# Split your data into in-control and out-of-control periods
in_control_sp = CUSUM_data_average_day[:pre_change_days]
out_control_sp = CUSUM_data_average_day[pre_change_days:total_days]

# Compute the mean and standard deviation for in-control and out-of-control periods
mu_in = np.mean(in_control_sp)
mu_out = np.mean(out_control_sp)
in_std = np.std(in_control_sp)


In [ ]:
summary_df = pd.DataFrame(columns=["Threshold", "False Positives", "True Positives", "Average Detection Delay", "MTBFA", "False Alarm Rate"])

for k_th in k_values:
    k = k_th * in_std
    h = control_limit * in_std  # h equal to 4*std

    # Call the CUSUM function
    signal, S_hi, S_lo = CUSUM(CUSUM_data_average_day, mu_in, k, h)
    # Plot
    fig, ax = plt.subplots(figsize=(15, 6))

    ax.plot(S_hi, label='High Side CUSUM', color='blue')
    ax.plot(S_lo, label='Low Side CUSUM', color='green')
    ax.axhline(y=h, color='black', linestyle='--', linewidth=2, label='Threshold (+h)')
    ax.axhline(y=-h, color='black', linestyle='--', linewidth=2, label='Threshold (-h)')
    ax.scatter(signal, [S_hi[i] for i in signal], color='black', zorder=5, label='Detected Shift')
    ax.scatter(signal, [S_lo[i] for i in signal], color='black', zorder=5)

    # Adding vertical lines for expected shift points (every 30 days starting from day 30)
    for day in range(30, total_days+1, 60):
      ax.axvline(x=day, color='purple', linestyle='--', label='Expected Shift Point' if day == 30 else "")

    #ax.set_title(f'Processing for k = {k}')
    ax.set_facecolor('white')  # White background

    ax.set_xlabel('Image Sequence')
    ax.set_ylabel('CUSUM Value')
    ax.legend()
    ax.grid(True, color='lightgrey')  # Black grid lines
    plt.show()

     # Calculate False Positives
    for i in range(pre_change_days):
        if S_hi[i] > h or S_lo[i] > -h:  # Assuming symmetry around zero for S_lo
            FalsePos.append(i + 1)
            DetectionTimes.append(i + 1)

    # Calculate True Positives and Detection Delay
    for i in range(pre_change_days, total_days):
        if S_hi[i] > h or S_lo[i] > -h:
            TruePos.append(i + 1)
            AvgDD.append(i + 1 - pre_change_days)
            break  # Remove this break if you want to count all true positives
